In [1]:
from data_prep import *
from PositionModel import *
from PlayerModel import *
from GeneralModel import *
from commons import *

In [2]:
features = ['goals_scored_avg5', 'xG_avg5', 'assists_fpl_avg5',
       'bonus_avg5', 'bps_avg5', 'clean_sheets_avg5', 'goals_conceded_avg5',
       'shots_avg5', 'xGBuildup_avg5', 'xGChain_avg5', 'yellow_cards_avg5',
       'red_cards_avg5', 'was_home', 'rating_diff', 'minutes_avg5']
rolling_columns = ['goals_scored', 'xG', 'assists_fpl', 'bonus', 'bps', 'clean_sheets', 'goals_conceded', 'shots', 'xGBuildup', 'xGChain', 'yellow_cards', 'red_cards', 'minutes']
to_predict = 'total_points'

In [3]:
# loading dataset
df = merged_understat_and_fpl(rolling_columns, True)
# df = load_dataset()

D:\PulpitE\FPL_ML\data_prep.py:86: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  fpl_data = pd.read_csv('fpl_players.csv')


Unnamed: 0                              1
season                               2016
name                         Aaron Lennon
position                              MID
team                                  NaN
assists                                 0
bonus                                   0
bps                                     6
clean_sheets                            0
creativity                            0.3
element                               142
fixture                                 3
goals_conceded                          0
goals_scored                            0
ict_index                             0.9
influence                             8.2
kickoff_time         2016-08-13T14:00:00Z
minutes                                15
opponent_team                          17
opp_team_name                       Spurs
own_goals                               0
penalties_missed                        0
penalties_saved                         0
red_cards                         

In [4]:
df[df['name'] == 'Harry Kane']

,Unnamed: 0,season,name,position_fpl,team,assists_fpl,bonus,bps,clean_sheets,creativity,...,xGChain_avg5,yellow_cards_avg5,red_cards_avg5,minutes_avg5,was_played,next_gw,opp_team,team_rating,opp_team_rating,rating_diff
50,51280,2020,Harry Kane,FWD,Spurs,0,0,7,0,22.5,...,0.905124,0.2,0.0,90.0,1,0,Brighton,1703.791260,1779.396606,-0.042489
49,51748,2020,Harry Kane,FWD,Spurs,4,3,81,0,40.8,...,0.609466,0.0,0.0,85.4,1,0,Leeds,1703.791260,1703.118164,0.000395
51,53420,2020,Harry Kane,FWD,Spurs,1,3,69,0,29.3,...,0.785562,0.0,0.0,90.0,1,0,Norwich,1703.791260,1577.265503,0.080218
50,54085,2020,Harry Kane,FWD,Spurs,1,0,17,1,12.3,...,0.548094,0.0,0.0,85.4,1,0,Brighton,1703.791260,1779.396606,-0.042489
52,54675,2020,Harry Kane,FWD,Spurs,0,0,18,0,3.1,...,0.739385,0.0,0.0,90.0,1,0,Chelsea,1703.791260,1853.123535,-0.080584
51,56476,2020,Harry Kane,FWD,Spurs,0,0,3,1,25.3,...,0.530702,0.0,0.0,88.6,1,0,Norwich,1703.791260,1577.265503,0.080218
53,57048,2020,Harry Kane,FWD,Spurs,1,3,39,1,11.2,...,0.503821,0.0,0.0,90.0,1,0,Liverpool,1703.791260,1982.519897,-0.140593
52,57623,2020,Harry Kane,FWD,Spurs,0,3,30,0,14.8,...,0.595488,0.0,0.0,88.6,1,0,Chelsea,1703.791260,1853.123535,-0.080584
53,58230,2020,Harry Kane,FWD,Spurs,0,0,0,0,0.8,...,0.647977,0.0,0.0,88.6,1,0,Liverpool,1703.791260,1982.519897,-0.140593
54,58838,2020,Harry Kane,FWD,Spurs,0,0,7,0,8.3,...,0.376784,0.0,0.0,90.0,1,0,West Ham,2030.751709,1750.152588,0.160328


In [5]:
# general model
print('GENERAL MODEL:')
X_train, X_test, y_train, y_test = split_data(df, features, to_predict)
general_model = GeneralModel()
general_model.fit(X_train, y_train)
y_pred = general_model.predict(X_test)
metrics = get_metrics(y_pred, y_test)
print(metrics)

GENERAL MODEL:
{'mse': 6.958101785714287, 'mae': 1.484964285714286, 'r2': 0.5360379985598158}


In [6]:
# position model (MID)
print('POSITION MODEL:')
df_mid = df[df['position_fpl'] == 'MID']
X_train, X_test, y_train, y_test = split_data(df_mid, features, to_predict)
position_model = PositionModel()
position_model.mid_model.fit(X_train, y_train)
y_pred = position_model.mid_model.predict(X_test)
metrics = get_metrics(y_pred, y_test)
print(metrics)

POSITION MODEL:
{'mse': 11.114998891966758, 'mae': 2.067423822714681, 'r2': 0.32247920894980653}


In [7]:
# salah only
print('PLAYER MODEL:')
salah = df[df['name'] == 'Mohamed Salah']
X_train, X_test, y_train, y_test = split_data(salah, features, to_predict, 0.8)
position_model = PlayerModel()
position_model.fit(X_train, y_train)
y_pred = position_model.predict(X_test)
metrics = get_metrics(y_pred, y_test)
print(metrics)

PLAYER MODEL:
{'mse': 20.61299, 'mae': 3.781, 'r2': -0.9354920187793427}


In [8]:
salah.tail(7)[['shots', 'shots_avg5', 'opp_team', 'goals', 'kickoff_time']]

,shots,shots_avg5,opp_team,goals,kickoff_time
742,7,4.4,Brighton,1,2022-03-12T12:30:00Z
743,3,4.0,Arsenal,0,2022-03-16T20:15:00Z
746,3,5.4,Watford,0,2022-04-02T11:30:00Z
747,2,5.0,Everton,0,2022-04-24T15:30:00Z
744,1,3.2,Aston Villa,0,2022-05-10T19:00:00Z
745,2,2.8,Fulham,1,2022-08-06T11:30:00Z
751,3,4.0,Crystal Palace,0,2022-08-15T19:00:00Z
